In [ ]:
cropTime = False
min_date = "2001-01-01 00:00:01"
max_date = "2001-01-01 00:00:01"
doubleAxis = True

from sklearn.metrics import mean_squared_error
from math import sqrt

# Plots
import matplotlib
import seaborn as sns
from matplotlib import gridspec
import plotly as ply
import plotly.graph_objs as go
import plotly.io as pio
pio.renderers.default = "jupyterlab"

sns.set(color_codes=True)
matplotlib.style.use('seaborn-whitegrid')

out_bbplot = widgets.Output()

def show_devices(Source):
    A_device.options = [s for s in list(records.readings[Source]['devices'].keys())]
    A_device.source = Source
    B_device.options = [s for s in list(records.readings[Source]['devices'].keys())]
    B_device.source = Source
    #_min_date.value = readings[Source].index.min()._short_repr
    #_max_date.value = readings[Source].index.max()._short_repr
    

def show_sensors_A(Source):
    A_sensor_drop.options = [s for s in list(records.readings[_test.value]['devices'][Source]['data'].columns)]
    A_sensor_drop.source = Source
    minCropDate.value = records.readings[_test.value]['devices'][Source]['data'].index.min()._short_repr
    maxCropDate.value = records.readings[_test.value]['devices'][Source]['data'].index.max()._short_repr
    
def show_sensors_B(Source):
    B_sensor_drop.options = [s for s in list(records.readings[_test.value]['devices'][Source]['data'].columns)]
    B_sensor_drop.source = Source
    minCropDate.value = records.readings[_test.value]['devices'][Source]['data'].index.min()._short_repr
    maxCropDate.value = records.readings[_test.value]['devices'][Source]['data'].index.max()._short_repr    

def redraw(b):
    with out_bbplot:
        cropTime = cropTimeCheck.value
        doubleAxis = doubleAxisCheck.value
        min_date = minCropDate.value
        max_date = maxCropDate.value
        mergedData = pd.merge(records.readings[_test.value]['devices'][A_device.value]['data'].loc[:,(A_sensor_drop.value,)], 
                              records.readings[_test.value]['devices'][B_device.value]['data'].loc[:,(B_sensor_drop.value,)], 
                              left_index=True, right_index=True, suffixes=('_' + A_sensor_drop.value, '_' + B_sensor_drop.value))
        clear_output()
        
        if cropTime:
            mergedData = mergedData[mergedData.index > min_date]
            mergedData = mergedData[mergedData.index < max_date]
            
        #jointplot
        df = pd.DataFrame()
        A = A_sensor_drop.value + '-' + A_device.value
        B = B_sensor_drop.value + '-' + B_device.value
        df[A] = mergedData.iloc[:,0]
        df[B] = mergedData.iloc[:,1]
        
        sns.set(font_scale=1.3)
        sns.set_style("whitegrid")
        sns.jointplot(A, B, data=df, kind="reg", color="b", height=12, scatter_kws={"s": 80})
        
        print ("data from " + str(df.index.min()) + " to " + str(df.index.max()))                      
        pearsonCorr = list(df.corr('pearson')[list(df.columns)[0]])[-1]
        rmse = sqrt(mean_squared_error(df[A].fillna(0), df[B].fillna(0)))
        
        print ('Pearson correlation coefficient: ' + str(pearsonCorr))
        print ('Coefficient of determination R²: ' + str(pearsonCorr*pearsonCorr))
        print ('RMSE: ' + str(rmse))
        print ('')
    
        if cropTime: 
            
            if (doubleAxis):
                layout = go.Layout(
                    #legend=dict(x=-.1, y=1.2), 
                    xaxis=dict(range=[min_date, max_date],title='Time'), 
                    yaxis=dict(zeroline=True, title=A, titlefont=dict(color='rgb(0,97,255)'), tickfont=dict(color='rgb(0,97,255)')),
                    yaxis2=dict(title=B,titlefont=dict(color='rgb(255,165,0)'), tickfont=dict(color='rgb(255,165,0)'), overlaying='y', side='right')
                )
            else:
                layout = go.Layout(
                    #legend=dict(x=-.1, y=1.2), 
                    xaxis=dict(range=[min_date, max_date],title='Time'), 
                    yaxis=dict(zeroline=True, title=A, titlefont=dict(color='rgb(0,97,255)'), tickfont=dict(color='rgb(0,97,255)')),
                )
                
        else:
            if (doubleAxis):
                layout = go.Layout(
                    #legend=dict(x=-.1, y=1.2), 
                    xaxis=dict(title='Time'), 
                    yaxis=dict(title=A, titlefont=dict(color='rgb(0,97,255)'), tickfont=dict(color='rgb(0,97,255)')),
                    yaxis2=dict(title=B, titlefont=dict(color='rgb(255,165,0)'), tickfont=dict(color='rgb(255,165,0)'), overlaying='y', side='right')
                )
            else:
                layout = go.Layout(
                    #legend=dict(x=-.1, y=1.2), 
                    xaxis=dict(title='Time'), 
                    yaxis=dict(zeroline=True, title=A, titlefont=dict(color='rgb(0,97,255)'), tickfont=dict(color='rgb(0,97,255)')),
                )
        difference = differenceCheck.value
        
        if not difference:
            trace0 = go.Scatter(x=df[A].index, y=df[A], name = A ,line = dict(color='rgb(0,97,255)'))
            
            if (doubleAxis):
                trace1 = go.Scatter(x=df[B].index,y=df[B],name=B, yaxis='y2', line = dict(color='rgb(255,165,0)'))
            else:
                trace1 = go.Scatter(x=df[B].index,y=df[B],name=B, line = dict(color='rgb(255,165,0)'))
            data = [trace0, trace1]
        else:
            trace0 = go.Scatter(x=df[A].index, y=df[A] - df[B], name = A + ' - ' + B, line = dict(color='rgb(0,97,255)'))
            data = [trace0]
        
        figure = go.Figure(data=data, layout=layout)
        ply.offline.iplot(figure)
        display(HTML('<hr>'))
        
        # Nice copy figure
        
        fig = plt.figure(figsize = (20,6))
        gs = gridspec.GridSpec(1, 3, figure=fig)
        
        ax1 = fig.add_subplot(gs[0, :-1])
        ax2 = fig.add_subplot(gs[0, -1])
        # identical to ax1 = plt.subplot(gs.new_subplotspec((0, 0), colspan=3))
    
        ax1.plot(df[A].index, df[A],'g', label = A, linewidth = 1, alpha = 0.9)
        ax1.plot(df[B].index, df[B], 'k', label = B, linewidth = 1, alpha = 0.7)
        ax1.axis('tight')
        ax1.set_title('Time Series Plot for {}'.format(text_title.value))
        ax1.grid(True)
        ax1.legend(loc="best")
        ax1.set_xlabel('Date (-)')
        ax1.set_ylabel(text_axis.value)
        ax1.set_ylim([0, 170])
        
        ax2.plot(df[B], df[A], 'go', label = A, linewidth = 1,  alpha = 0.3)
        ax2.plot(df[B], df[B], 'k', label =  '1:1 Line', linewidth = 0.4, alpha = 0.3)
        ax2.axis('tight')
        ax2.set_title('Scatter Plot for {}'.format(text_title.value))
        ax2.grid(True)
        ax2.legend(loc="best")
        ax2.set_xlabel(text_axis.value)
        ax2.set_ylabel(text_axis.value)
        ax2.set_xlim([0, 60])
        ax2.set_ylim([0, 60])
        
        if coherencePlot.value:
            dt = 1
            fig = plt.figure(figsize = (20,6))
            # axs[0].plot(t, s1, t, s2)
            # axs[0].set_xlim(0, 2)
            # axs[0].set_xlabel('time')
            # axs[0].set_ylabel('s1 and s2')
            # axs[0].grid(True)
            
            cxy, f = plt.cohere(df[A], df[B], 512, 2, detrend = 'linear')
            plt.ylabel('Coherence')

            fig.tight_layout()
            
        plt.show()
        
if len(records.readings) < 1: 
    with out_bbplot:
        print ("Please load some data first...")
else:
    layout=widgets.Layout(width='350px')
    b_redraw = widgets.Button(description='Draw')
    b_redraw.on_click(redraw)
    doubleAxisCheck = widgets.Checkbox(value=False, description='Secondary y axis', disabled=False)
    
    cropTimeCheck = widgets.Checkbox(value=False,description='Crop Data in X axis', disabled=False)
    minCropDate = widgets.Text(description='Start date:', layout=layout)
    maxCropDate = widgets.Text(description='End date:', layout=layout)
    
    # Test dropdown
    _test = widgets.Dropdown(options=[k for k in records.readings.keys()], 
                        layout = widgets.Layout(width='350px'),
                        description = 'Test')
    
    _test_drop = widgets.interactive(show_devices, 
                                Source=_test,)
    
    # Device dropdown
    A_device = widgets.Dropdown(layout=layout,
                            description = 'Device 1')
    
    A_device_drop = widgets.interactive(show_sensors_A, 
                                    Source=A_device, 
                                    layout=layout)
    
    B_device = widgets.Dropdown(layout=layout,
                            description = 'Device 2')
    
    B_device_drop = widgets.interactive(show_sensors_B, 
                                    Source=B_device, 
                                    layout=layout)
    
    # Sensor dropdown
    A_sensor_drop = widgets.Dropdown(layout=layout,
                               description = 'Channel 1')
    
    # Sensor dropdown
    B_sensor_drop = widgets.Dropdown(layout=layout,
                               description = 'Channel 2')
    
    # Synchronise Checkbox
    differenceCheck = widgets.Checkbox(value=False, 
                                 description='Time Series Difference', 
                                 disabled=False, 
                                 layout=widgets.Layout(width='300px'))
    
    coherencePlot = widgets.Checkbox(value=False, 
                                 description='Coherence plot', 
                                 disabled=False, 
                                 layout=widgets.Layout(width='300px'))
    
    draw_box = widgets.HBox([doubleAxisCheck, differenceCheck, coherencePlot])
    test_box = widgets.HBox([_test_drop], layout = widgets.Layout(width='400px'))
    device_box = widgets.HBox([A_device, B_device])
    sensor_box = widgets.HBox([A_sensor_drop, B_sensor_drop])
    crop_box = widgets.HBox([minCropDate, maxCropDate, cropTimeCheck])
    button_box = widgets.VBox([b_redraw])
    text_axis = widgets.Text(description='Text axis', layout=layout)
    text_title = widgets.Text(description='Text title', layout=layout)
    root_box = widgets.VBox([draw_box, test_box, device_box, sensor_box, crop_box, button_box, text_axis, text_title])
        
    display(root_box)
    display(out_bbplot)